# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
DATA_PATH = "../data"

In [3]:
df_train = pd.read_csv(f"{DATA_PATH}/train.csv")
df_test = pd.read_csv(f"{DATA_PATH}/test.csv")

# Pre processing

In order to proceed with futher analysis, I will divide the training dataset into two: before the vaccine and after the vaccine. 

In [4]:
df_train

,Id,Location,Weekly Cases,Year,Weekly Cases per Million,Weekly Deaths,Weekly Deaths per Million,Total Vaccinations,People Vaccinated,People Fully Vaccinated,Total Boosters,Daily Vaccinations,Total Vaccinations per Hundred,People Vaccinated per Hundred,People Fully Vaccinated per Hundred,Total Boosters per Hundred,Daily Vaccinations per Hundred,Daily People Vaccinated,Daily People Vaccinated per Hundred,Next Week's Deaths
0,911530868,World,2372.0,2020,0.300,65.0,0.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,344.0
1,807936902,World,5023.0,2020,0.635,114.0,0.014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,361.0
2,773590408,World,5612.0,2020,0.710,116.0,0.015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,431.0
3,130466459,World,7580.0,2020,0.958,153.0,0.019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,463.0
4,544040446,World,8983.0,2020,1.136,187.0,0.024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,506.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129151,541829605,Zimbabwe,464.0,2022,29.012,13.0,0.813,11949993.0,6297324.0,4601845.0,1050824.0,5665.0,74.72,39.37,28.77,6.57,354.0,1427.0,0.009,9.0
129152,969939474,Zimbabwe,471.0,2022,29.449,12.0,0.750,11958771.0,6299348.0,4605821.0,1053602.0,5295.0,74.77,39.39,28.80,6.59,331.0,1362.0,0.009,7.0
129153,667902340,Zimbabwe,450.0,2022,28.136,13.0,0.813,NaN,NaN,NaN,NaN,5316.0,NaN,NaN,NaN,NaN,332.0,1483.0,0.009,5.0
129154,961193720,Zimbabwe,277.0,2022,17.320,6.0,0.375,11974313.0,6305470.0,4611113.0,1057730.0,5358.0,74.87,39.43,28.83,6.61,335.0,1633.0,0.010,7.0


In [5]:
df_test

,Id,Location,Weekly Cases,Year,Weekly Cases per Million,Weekly Deaths,Weekly Deaths per Million,Total Vaccinations,People Vaccinated,People Fully Vaccinated,Total Boosters,Daily Vaccinations,Total Vaccinations per Hundred,People Vaccinated per Hundred,People Fully Vaccinated per Hundred,Total Boosters per Hundred,Daily Vaccinations per Hundred,Daily People Vaccinated,Daily People Vaccinated per Hundred
0,719557359,Monaco,185.0,2021,5042.796,0.0,0.000,NaN,NaN,NaN,NaN,158.0,NaN,NaN,NaN,NaN,4307.0,23.0,0.063
1,507527289,Liechtenstein,78.0,2020,1998.002,0.0,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,821551528,Albania,230.0,2022,80.569,1.0,0.350,NaN,NaN,NaN,NaN,1462.0,NaN,NaN,NaN,NaN,512.0,391.0,0.014
3,696602016,Niger,14.0,2022,0.554,1.0,0.040,NaN,NaN,NaN,NaN,72018.0,NaN,NaN,NaN,NaN,2852.0,60442.0,0.239
4,680387432,French Polynesia,1033.0,2022,3397.669,0.0,0.000,366984.0,186177.0,180807.0,NaN,22.0,120.71,61.24,59.47,NaN,72.0,22.0,0.007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43047,713850901,Luxembourg,585.0,2021,915.033,0.0,0.000,766418.0,408405.0,NaN,455.0,991.0,119.88,63.88,NaN,0.07,1550.0,466.0,0.073
43048,980183629,Colombia,18120.0,2020,351.732,584.0,11.336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43049,386597907,United Kingdom,38399.0,2021,570.725,592.0,8.799,30691557.0,28327873.0,2363684.0,NaN,598286.0,45.62,42.10,3.51,NaN,8892.0,498281.0,0.741
43050,293365001,Cyprus,22.0,2020,24.553,0.0,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_b4v = df_train.loc[df_train["Total Vaccinations"].isna() & df_train["Daily Vaccinations"].isna()] # Also checking the daily vaccinations so we don't include the first day of vaccination
df_b4v

,Id,Location,Weekly Cases,Year,Weekly Cases per Million,Weekly Deaths,Weekly Deaths per Million,Total Vaccinations,People Vaccinated,People Fully Vaccinated,Total Boosters,Daily Vaccinations,Total Vaccinations per Hundred,People Vaccinated per Hundred,People Fully Vaccinated per Hundred,Total Boosters per Hundred,Daily Vaccinations per Hundred,Daily People Vaccinated,Daily People Vaccinated per Hundred,Next Week's Deaths
0,911530868,World,2372.0,2020,0.300,65.0,0.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,344.0
1,807936902,World,5023.0,2020,0.635,114.0,0.014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,361.0
2,773590408,World,5612.0,2020,0.710,116.0,0.015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,431.0
3,130466459,World,7580.0,2020,0.958,153.0,0.019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,463.0
4,544040446,World,8983.0,2020,1.136,187.0,0.024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,506.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128747,713525252,Zimbabwe,617.0,2021,38.578,82.0,5.127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0
128748,180549162,Zimbabwe,620.0,2021,38.766,74.0,4.627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0
128749,949437505,Zimbabwe,564.0,2021,35.264,71.0,4.439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0
128750,451524791,Zimbabwe,534.0,2021,33.389,61.0,3.814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0


In [7]:
df_b4v.to_csv(f"{DATA_PATH}/train_b4v.csv", index=False)

In [8]:
df_av = df_train[~df_train.isin(df_b4v)].dropna(how="all")
df_av

,Id,Location,Weekly Cases,Year,Weekly Cases per Million,Weekly Deaths,Weekly Deaths per Million,Total Vaccinations,People Vaccinated,People Fully Vaccinated,Total Boosters,Daily Vaccinations,Total Vaccinations per Hundred,People Vaccinated per Hundred,People Fully Vaccinated per Hundred,Total Boosters per Hundred,Daily Vaccinations per Hundred,Daily People Vaccinated,Daily People Vaccinated per Hundred,Next Week's Deaths
221,563444119.0,World,4148513.0,2020.0,524.511,73380.0,9.278,0.0,0.0,NaN,NaN,0.0,0.00,0.00,NaN,NaN,0.0,0.0,0.000,77091.0
222,958550508.0,World,4246307.0,2020.0,536.876,75053.0,9.489,0.0,0.0,NaN,NaN,1.0,0.00,0.00,NaN,NaN,0.0,1.0,0.000,76922.0
223,468761248.0,World,4248127.0,2020.0,537.106,75971.0,9.605,1.0,1.0,NaN,NaN,1.0,0.00,0.00,NaN,NaN,0.0,1.0,0.000,77648.0
224,993915047.0,World,4284262.0,2020.0,541.674,77106.0,9.749,1.0,1.0,NaN,NaN,1.0,0.00,0.00,NaN,NaN,0.0,1.0,0.000,77953.0
225,148844298.0,World,4333098.0,2020.0,547.849,77629.0,9.815,1.0,1.0,NaN,NaN,1.0,0.00,0.00,NaN,NaN,0.0,1.0,0.000,78095.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129151,541829605.0,Zimbabwe,464.0,2022.0,29.012,13.0,0.813,11949993.0,6297324.0,4601845.0,1050824.0,5665.0,74.72,39.37,28.77,6.57,354.0,1427.0,0.009,9.0
129152,969939474.0,Zimbabwe,471.0,2022.0,29.449,12.0,0.750,11958771.0,6299348.0,4605821.0,1053602.0,5295.0,74.77,39.39,28.80,6.59,331.0,1362.0,0.009,7.0
129153,667902340.0,Zimbabwe,450.0,2022.0,28.136,13.0,0.813,NaN,NaN,NaN,NaN,5316.0,NaN,NaN,NaN,NaN,332.0,1483.0,0.009,5.0
129154,961193720.0,Zimbabwe,277.0,2022.0,17.320,6.0,0.375,11974313.0,6305470.0,4611113.0,1057730.0,5358.0,74.87,39.43,28.83,6.61,335.0,1633.0,0.010,7.0


In [9]:
df_av.to_csv(f"{DATA_PATH}/train_av.csv", index=False)

## Column type definition

In [10]:
df_b4v.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51172 entries, 0 to 128751
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Id                                   51172 non-null  int64  
 1   Location                             51172 non-null  object 
 2   Weekly Cases                         51016 non-null  float64
 3   Year                                 51172 non-null  int64  
 4   Weekly Cases per Million             50361 non-null  float64
 5   Weekly Deaths                        50064 non-null  float64
 6   Weekly Deaths per Million            49413 non-null  float64
 7   Total Vaccinations                   0 non-null      float64
 8   People Vaccinated                    78 non-null     float64
 9   People Fully Vaccinated              68 non-null     float64
 10  Total Boosters                       4 non-null      float64
 11  Daily Vaccinations         

In [11]:
df_av.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77984 entries, 221 to 129155
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Id                                   77984 non-null  float64
 1   Location                             77984 non-null  object 
 2   Weekly Cases                         77927 non-null  float64
 3   Year                                 77984 non-null  float64
 4   Weekly Cases per Million             77927 non-null  float64
 5   Weekly Deaths                        77834 non-null  float64
 6   Weekly Deaths per Million            77834 non-null  float64
 7   Total Vaccinations                   40270 non-null  float64
 8   People Vaccinated                    38351 non-null  float64
 9   People Fully Vaccinated              36506 non-null  float64
 10  Total Boosters                       19843 non-null  float64
 11  Daily Vaccinations       

In [12]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43052 entries, 0 to 43051
Data columns (total 19 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Id                                   43052 non-null  int64  
 1   Location                             43052 non-null  object 
 2   Weekly Cases                         42979 non-null  float64
 3   Year                                 43052 non-null  int64  
 4   Weekly Cases per Million             42768 non-null  float64
 5   Weekly Deaths                        42603 non-null  float64
 6   Weekly Deaths per Million            42395 non-null  float64
 7   Total Vaccinations                   13409 non-null  float64
 8   People Vaccinated                    12822 non-null  float64
 9   People Fully Vaccinated              12259 non-null  float64
 10  Total Boosters                       6555 non-null   float64
 11  Daily Vaccinations          

The column "Location" is type object, it will be changed to type category. Also, the columns "Weekly Cases", "Weekly Deaths", "Total Vaccinations", "People Vaccinated", "People Fully Vaccinated", "Total Boosters", "Daily Vaccinations", "Daily People Vaccinated" should be type int but we cannot convert it since there are null values on them.

In [13]:
df_b4v["Location"] = df_b4v["Location"].astype('category')
df_b4v.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51172 entries, 0 to 128751
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Id                                   51172 non-null  int64   
 1   Location                             51172 non-null  category
 2   Weekly Cases                         51016 non-null  float64 
 3   Year                                 51172 non-null  int64   
 4   Weekly Cases per Million             50361 non-null  float64 
 5   Weekly Deaths                        50064 non-null  float64 
 6   Weekly Deaths per Million            49413 non-null  float64 
 7   Total Vaccinations                   0 non-null      float64 
 8   People Vaccinated                    78 non-null     float64 
 9   People Fully Vaccinated              68 non-null     float64 
 10  Total Boosters                       4 non-null      float64 
 11  Daily Vaccinat

C:\Users\bianc\AppData\Local\Temp\ipykernel_4916\2011787304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b4v["Location"] = df_b4v["Location"].astype('category')


In [14]:
df_av["Location"] = df_av["Location"].astype('category')
df_av.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77984 entries, 221 to 129155
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Id                                   77984 non-null  float64 
 1   Location                             77984 non-null  category
 2   Weekly Cases                         77927 non-null  float64 
 3   Year                                 77984 non-null  float64 
 4   Weekly Cases per Million             77927 non-null  float64 
 5   Weekly Deaths                        77834 non-null  float64 
 6   Weekly Deaths per Million            77834 non-null  float64 
 7   Total Vaccinations                   40270 non-null  float64 
 8   People Vaccinated                    38351 non-null  float64 
 9   People Fully Vaccinated              36506 non-null  float64 
 10  Total Boosters                       19843 non-null  float64 
 11  Daily Vaccin

In [15]:
df_test["Location"] = df_test["Location"].astype('category')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43052 entries, 0 to 43051
Data columns (total 19 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Id                                   43052 non-null  int64   
 1   Location                             43052 non-null  category
 2   Weekly Cases                         42979 non-null  float64 
 3   Year                                 43052 non-null  int64   
 4   Weekly Cases per Million             42768 non-null  float64 
 5   Weekly Deaths                        42603 non-null  float64 
 6   Weekly Deaths per Million            42395 non-null  float64 
 7   Total Vaccinations                   13409 non-null  float64 
 8   People Vaccinated                    12822 non-null  float64 
 9   People Fully Vaccinated              12259 non-null  float64 
 10  Total Boosters                       6555 non-null   float64 
 11  Daily Vaccinati

## Data cleaning and Treating missing data

### Checking before the vaccine

In [16]:
df_b4v.notna().sum()

Id                                     51172
Location                               51172
Weekly Cases                           51016
Year                                   51172
Weekly Cases per Million               50361
Weekly Deaths                          50064
Weekly Deaths per Million              49413
Total Vaccinations                         0
People Vaccinated                         78
People Fully Vaccinated                   68
Total Boosters                             4
Daily Vaccinations                         0
Total Vaccinations per Hundred             0
People Vaccinated per Hundred             78
People Fully Vaccinated per Hundred       68
Total Boosters per Hundred                 4
Daily Vaccinations per Hundred             0
Daily People Vaccinated                  192
Daily People Vaccinated per Hundred      192
Next Week's Deaths                     51172
dtype: int64

There are 78 occurences of People Vaccinated, that should not be happening.

In [17]:
df_b4v.loc[df_b4v["People Vaccinated"].notna()]

,Id,Location,Weekly Cases,Year,Weekly Cases per Million,Weekly Deaths,Weekly Deaths per Million,Total Vaccinations,People Vaccinated,People Fully Vaccinated,Total Boosters,Daily Vaccinations,Total Vaccinations per Hundred,People Vaccinated per Hundred,People Fully Vaccinated per Hundred,Total Boosters per Hundred,Daily Vaccinations per Hundred,Daily People Vaccinated,Daily People Vaccinated per Hundred,Next Week's Deaths
31222,343994044,Cuba,8473.0,2021,752.729,72.0,6.396,NaN,455908.0,NaN,NaN,NaN,NaN,4.05,NaN,NaN,NaN,96477.0,0.857,73.0
31223,912487837,Cuba,8414.0,2021,747.488,73.0,6.485,NaN,540002.0,NaN,NaN,NaN,NaN,4.80,NaN,NaN,NaN,94000.0,0.835,76.0
31224,573668672,Cuba,8587.0,2021,762.857,71.0,6.308,NaN,612376.0,NaN,NaN,NaN,NaN,5.44,NaN,NaN,NaN,90396.0,0.803,75.0
31226,294990570,Cuba,8785.0,2021,780.447,62.0,5.508,NaN,717997.0,NaN,NaN,NaN,NaN,6.38,NaN,NaN,NaN,78789.0,0.700,81.0
31227,754881757,Cuba,8611.0,2021,764.989,67.0,5.952,NaN,774212.0,NaN,NaN,NaN,NaN,6.88,NaN,NaN,NaN,59254.0,0.526,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64153,187911362,Kazakhstan,333.0,2022,17.347,1.0,0.052,NaN,9522541.0,9304087.0,NaN,NaN,NaN,49.61,48.47,NaN,NaN,495.0,0.003,0.0
64155,898399739,Kazakhstan,613.0,2022,31.933,1.0,0.052,NaN,9523631.0,9306077.0,NaN,NaN,NaN,49.61,48.48,NaN,NaN,560.0,0.003,0.0
64157,188347305,Kazakhstan,707.0,2022,36.830,0.0,0.000,NaN,9524896.0,9308744.0,NaN,NaN,NaN,49.62,48.49,NaN,NaN,524.0,0.003,0.0
64158,641884893,Kazakhstan,645.0,2022,33.600,0.0,0.000,NaN,9525645.0,9310076.0,NaN,NaN,NaN,49.62,48.50,NaN,NaN,629.0,0.003,0.0


These occurences will be removed.

In [18]:
df_b4v = df_b4v.loc[df_b4v["People Vaccinated"].isna()]
df_b4v.notna().sum()

Id                                     51094
Location                               51094
Weekly Cases                           50938
Year                                   51094
Weekly Cases per Million               50283
Weekly Deaths                          49986
Weekly Deaths per Million              49335
Total Vaccinations                         0
People Vaccinated                          0
People Fully Vaccinated                    2
Total Boosters                             0
Daily Vaccinations                         0
Total Vaccinations per Hundred             0
People Vaccinated per Hundred              0
People Fully Vaccinated per Hundred        2
Total Boosters per Hundred                 0
Daily Vaccinations per Hundred             0
Daily People Vaccinated                  115
Daily People Vaccinated per Hundred      115
Next Week's Deaths                     51094
dtype: int64

I will also remove the occurences of People Fully Vaccinated, People Fully Vaccinated per Hundred, Daily People Vaccinated and Daily People Vaccinated per Hundred.

In [19]:
df_b4v = df_b4v.loc[df_b4v["People Fully Vaccinated"].isna() & df_b4v["People Fully Vaccinated per Hundred"].isna() & df_b4v["Daily People Vaccinated"].isna() & df_b4v["Daily People Vaccinated per Hundred"].isna()]
df_b4v.notna().sum()

Id                                     50979
Location                               50979
Weekly Cases                           50823
Year                                   50979
Weekly Cases per Million               50168
Weekly Deaths                          49871
Weekly Deaths per Million              49220
Total Vaccinations                         0
People Vaccinated                          0
People Fully Vaccinated                    0
Total Boosters                             0
Daily Vaccinations                         0
Total Vaccinations per Hundred             0
People Vaccinated per Hundred              0
People Fully Vaccinated per Hundred        0
Total Boosters per Hundred                 0
Daily Vaccinations per Hundred             0
Daily People Vaccinated                    0
Daily People Vaccinated per Hundred        0
Next Week's Deaths                     50979
dtype: int64

In [20]:
df_b4v.dropna(axis=1, how="all", inplace=True)

Now, lets see the occurrences of nulls.

In [21]:
df_b4v

,Id,Location,Weekly Cases,Year,Weekly Cases per Million,Weekly Deaths,Weekly Deaths per Million,Next Week's Deaths
0,911530868,World,2372.0,2020,0.300,65.0,0.008,344.0
1,807936902,World,5023.0,2020,0.635,114.0,0.014,361.0
2,773590408,World,5612.0,2020,0.710,116.0,0.015,431.0
3,130466459,World,7580.0,2020,0.958,153.0,0.019,463.0
4,544040446,World,8983.0,2020,1.136,187.0,0.024,506.0
...,...,...,...,...,...,...,...,...
128747,713525252,Zimbabwe,617.0,2021,38.578,82.0,5.127,34.0
128748,180549162,Zimbabwe,620.0,2021,38.766,74.0,4.627,36.0
128749,949437505,Zimbabwe,564.0,2021,35.264,71.0,4.439,31.0
128750,451524791,Zimbabwe,534.0,2021,33.389,61.0,3.814,34.0


Will not drop these columns for now since Next Week's Deaths will be used for the analysis.

In [22]:
df_b4v.to_csv(f"{DATA_PATH}/train_b4v.csv", index=False)

### Checking After the vaccine

In [23]:
df_av.isna().sum()

Id                                         0
Location                                   0
Weekly Cases                              57
Year                                       0
Weekly Cases per Million                  57
Weekly Deaths                            150
Weekly Deaths per Million                150
Total Vaccinations                     37714
People Vaccinated                      39633
People Fully Vaccinated                41478
Total Boosters                         58141
Daily Vaccinations                       144
Total Vaccinations per Hundred         37714
People Vaccinated per Hundred          39633
People Fully Vaccinated per Hundred    41478
Total Boosters per Hundred             58141
Daily Vaccinations per Hundred           144
Daily People Vaccinated                  785
Daily People Vaccinated per Hundred      785
Next Week's Deaths                         0
dtype: int64

Will not drop these columns for now since Next Week's Deaths will be used for the analysis.

## Normalization and discretization

Joining df_b4v and df_av.

In [25]:
df_train = pd.concat([df_b4v, df_av])
df_train

,Id,Location,Weekly Cases,Year,Weekly Cases per Million,Weekly Deaths,Weekly Deaths per Million,Next Week's Deaths,Total Vaccinations,People Vaccinated,People Fully Vaccinated,Total Boosters,Daily Vaccinations,Total Vaccinations per Hundred,People Vaccinated per Hundred,People Fully Vaccinated per Hundred,Total Boosters per Hundred,Daily Vaccinations per Hundred,Daily People Vaccinated,Daily People Vaccinated per Hundred
0,911530868.0,World,2372.0,2020.0,0.300,65.0,0.008,344.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,807936902.0,World,5023.0,2020.0,0.635,114.0,0.014,361.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,773590408.0,World,5612.0,2020.0,0.710,116.0,0.015,431.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,130466459.0,World,7580.0,2020.0,0.958,153.0,0.019,463.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,544040446.0,World,8983.0,2020.0,1.136,187.0,0.024,506.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129151,541829605.0,Zimbabwe,464.0,2022.0,29.012,13.0,0.813,9.0,11949993.0,6297324.0,4601845.0,1050824.0,5665.0,74.72,39.37,28.77,6.57,354.0,1427.0,0.009
129152,969939474.0,Zimbabwe,471.0,2022.0,29.449,12.0,0.750,7.0,11958771.0,6299348.0,4605821.0,1053602.0,5295.0,74.77,39.39,28.80,6.59,331.0,1362.0,0.009
129153,667902340.0,Zimbabwe,450.0,2022.0,28.136,13.0,0.813,5.0,NaN,NaN,NaN,NaN,5316.0,NaN,NaN,NaN,NaN,332.0,1483.0,0.009
129154,961193720.0,Zimbabwe,277.0,2022.0,17.320,6.0,0.375,7.0,11974313.0,6305470.0,4611113.0,1057730.0,5358.0,74.87,39.43,28.83,6.61,335.0,1633.0,0.010


Creating a new feature related to the vaccine existence.

In [26]:
df_train["After Vaccine"] = df_train["People Vaccinated"].apply(lambda x: False if pd.isna(x) else True)
df_train

,Id,Location,Weekly Cases,Year,Weekly Cases per Million,Weekly Deaths,Weekly Deaths per Million,Next Week's Deaths,Total Vaccinations,People Vaccinated,...,Total Boosters,Daily Vaccinations,Total Vaccinations per Hundred,People Vaccinated per Hundred,People Fully Vaccinated per Hundred,Total Boosters per Hundred,Daily Vaccinations per Hundred,Daily People Vaccinated,Daily People Vaccinated per Hundred,After Vaccine
0,911530868.0,World,2372.0,2020.0,0.300,65.0,0.008,344.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,807936902.0,World,5023.0,2020.0,0.635,114.0,0.014,361.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,773590408.0,World,5612.0,2020.0,0.710,116.0,0.015,431.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,130466459.0,World,7580.0,2020.0,0.958,153.0,0.019,463.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,544040446.0,World,8983.0,2020.0,1.136,187.0,0.024,506.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129151,541829605.0,Zimbabwe,464.0,2022.0,29.012,13.0,0.813,9.0,11949993.0,6297324.0,...,1050824.0,5665.0,74.72,39.37,28.77,6.57,354.0,1427.0,0.009,True
129152,969939474.0,Zimbabwe,471.0,2022.0,29.449,12.0,0.750,7.0,11958771.0,6299348.0,...,1053602.0,5295.0,74.77,39.39,28.80,6.59,331.0,1362.0,0.009,True
129153,667902340.0,Zimbabwe,450.0,2022.0,28.136,13.0,0.813,5.0,NaN,NaN,...,NaN,5316.0,NaN,NaN,NaN,NaN,332.0,1483.0,0.009,False
129154,961193720.0,Zimbabwe,277.0,2022.0,17.320,6.0,0.375,7.0,11974313.0,6305470.0,...,1057730.0,5358.0,74.87,39.43,28.83,6.61,335.0,1633.0,0.010,True


In [27]:
df_train["After Vaccine"].unique()

array([False,  True])

In [28]:
df_train.reset_index(inplace=True)

In [29]:
df_train.to_csv(f"{DATA_PATH}/train.csv")